# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


- ### Check CPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

# I. Toy examples

- ### Test

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [4]:
#load file
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 600901
total chars: 59


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [5]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [6]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 408 ms, sys: 32 ms, total: 440 ms
Wall time: 437 ms


** 2. Build the network**

In [7]:
HIDDEN_DIM= 128 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [8]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [9]:
# Generate some sample before training to know how bad it is!
generate_text(model, 100, VOCAB_SIZE, indices_char)

'��b4bbloooo,,!!!jjjjjmm]jj-____...'''mbmbmzzzzzz_=____fffbbmmzzzzzz==v____...bmzzzzzzz==_____..bbmz

"'\xc3\xc3b4bbloooo,,!!!jjjjjmm]jj-____...'''mbmbmzzzzzz_=____fffbbmmzzzzzz==v____...bmzzzzzzz==_____..bbmzz"

**3. Train network**

In [14]:
nb_iteration = 0
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=100


while True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    nb_iteration += 1
    generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if nb_iteration % 10 == 0:
        print("\n\nIteration nb : %s" %nb_iteration)
        model.save_weights('weight_01/checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_iteration))



--------------------
Epoch 1/1
11s - loss: 1.3599
s and soul and the same time of the standards of the standards of the standards of the standards of 

--------------------
Epoch 1/1
11s - loss: 1.3569
8. the conscience of the conscience of the conscience of the conscience of the conscience of the con

--------------------
Epoch 1/1
11s - loss: 1.3551
3. the same things to the same things to the same things to the same things to the same things to th

--------------------
Epoch 1/1
11s - loss: 1.3515
�self in the same that it is the same that it is the same that it is the same that it is the same th

--------------------
Epoch 1/1
11s - loss: 1.3481
 and more present and more present and more present and more present and more present and more prese

--------------------
Epoch 1/1
11s - loss: 1.3461
�����j it is the entertors of the endirated of the entertors of the endirated of the entertors of th

--------------------
Epoch 1/1
11s - loss: 1.3432
[complicated and the subtlection of 

KeyboardInterrupt: 

**4. Generate text**

In [ ]:
generate_text(model, 100, VOCAB_SIZE, ix_to_char)